# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Torbay,47.6666,-52.7314,28.99,100,90,12.66,CA,1612065409
1,1,Viedma,-40.8135,-62.9967,57.83,59,45,7.78,AR,1612065409
2,2,Cape Town,-33.9258,18.4232,68.00,77,75,9.22,ZA,1612065367
3,3,Atuona,-9.8000,-139.0333,79.12,76,88,17.20,PF,1612065409
4,4,Punta Arenas,-53.1500,-70.9167,48.20,71,0,24.16,CL,1612065409
...,...,...,...,...,...,...,...,...,...,...
568,568,Cockburn Town,21.4612,-71.1419,73.40,56,20,11.50,TC,1612065474
569,569,Burgkirchen,48.2044,13.0999,30.00,99,100,5.99,AT,1612065474
570,570,Sobolevo,54.4299,31.8996,18.09,97,100,6.02,RU,1612065474
571,571,Burgos,42.3394,-3.7079,37.99,87,35,14.97,ES,1612065474


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [13]:
print(gmaps.configure(api_key=g_key))

None


In [43]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=cities_df['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
ideal_weather = (cities_df['Wind Speed']< 10)  & (cities_df['Cloudiness'] ==0) &(cities_df['Max Temp'] >70) & (cities_df['Max Temp'] <80)
cities_df[ideal_weather].dropna()

#

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,61,Yāval,21.1667,75.7000,72.90,19,0,7.61,IN,1612065415
210,210,Sur,22.5667,59.5289,71.78,42,0,1.01,OM,1612065434
251,251,Jalingo,8.8833,11.3667,71.82,22,0,3.58,NG,1612065438
328,328,Beloha,-25.1667,45.0500,79.38,71,0,4.85,MG,1612065447
367,367,Coahuayana Viejo,18.7333,-103.6833,73.00,88,0,2.59,MX,1612065451
377,377,Tomatlán,19.9333,-105.2500,70.61,61,0,2.26,MX,1612065452
558,558,Ambilobe,-13.2000,49.0500,75.76,88,0,2.06,MG,1612065472


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df =cities_df[ideal_weather].copy()
hotel_df['Hotel Name'] =''
hotel_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
61,61,Yāval,21.1667,75.7000,72.90,19,0,7.61,IN,1612065415,
210,210,Sur,22.5667,59.5289,71.78,42,0,1.01,OM,1612065434,
251,251,Jalingo,8.8833,11.3667,71.82,22,0,3.58,NG,1612065438,
328,328,Beloha,-25.1667,45.0500,79.38,71,0,4.85,MG,1612065447,
367,367,Coahuayana Viejo,18.7333,-103.6833,73.00,88,0,2.59,MX,1612065451,
377,377,Tomatlán,19.9333,-105.2500,70.61,61,0,2.26,MX,1612065452,
558,558,Ambilobe,-13.2000,49.0500,75.76,88,0,2.06,MG,1612065472,


In [37]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [38]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
61,61,Yāval,21.1667,75.7000,72.90,19,0,7.61,IN,1612065415,Four Seasons Recreation Pvt Ltd
210,210,Sur,22.5667,59.5289,71.78,42,0,1.01,OM,1612065434,Sur Plaza Hotel
251,251,Jalingo,8.8833,11.3667,71.82,22,0,3.58,NG,1612065438,Shield Hotel
328,328,Beloha,-25.1667,45.0500,79.38,71,0,4.85,MG,1612065447,Restaurant Le Paradis du sud
367,367,Coahuayana Viejo,18.7333,-103.6833,73.00,88,0,2.59,MX,1612065451,Hotel Cuyutlan Holistico
377,377,Tomatlán,19.9333,-105.2500,70.61,61,0,2.26,MX,1612065452,Las Alamandas
558,558,Ambilobe,-13.2000,49.0500,75.76,88,0,2.06,MG,1612065472,Le Relais de l'Ankarana


In [48]:
#hotel_df

In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [52]:
# Convert hotel name to list
hotel_name = hotel_df["Hotel Name"].tolist()
# Create bank symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,info_box_content=hotel_info   
)

#info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]

#fig = gmaps.figure()
# fig.add_layer(hotel_layer)

# fig

# Add marker layer ontop of heat map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)



# Display figure
fig

Figure(layout=FigureLayout(height='420px'))